In [134]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta
import utils
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

In [135]:
events = utils.get_events()

In [136]:
events.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,city,region,country,device_type,screen_resolution,operating_system_version,browser_version,month,day,hora
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,18,0
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,18,0
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,18,0
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,18,0
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,18,0


In [137]:
events.columns

Index(['timestamp', 'event', 'person', 'url', 'sku', 'model', 'condition',
       'storage', 'color', 'skus', 'search_term', 'staticpage',
       'campaign_source', 'search_engine', 'channel', 'new_vs_returning',
       'city', 'region', 'country', 'device_type', 'screen_resolution',
       'operating_system_version', 'browser_version', 'month', 'day', 'hora'],
      dtype='object')

In [138]:
events['event'].values

[viewed product, viewed product, viewed product, viewed product, viewed product, ..., lead, lead, lead, lead, lead]
Length: 2341681
Categories (11, object): [ad campaign hit, brand listing, checkout, conversion, ..., searched products, staticpage, viewed product, visited site]

In [139]:
events['marca'] = events['model'].str.split(' ').str[0]
events['modelo'] = events['model'].str.split(' ').str[1].fillna('') + ' ' + events['model'].str.split(' ').str[2].fillna('')
events[['marca', 'modelo']].head()

,marca,modelo
0,Samsung,Galaxy J7
1,iPhone,5s
2,iPhone,6S
3,iPhone,7
4,LG,G4 H818P


In [140]:
usuarios_checkout = events.loc[events['event'] == 'checkout',['person','model','marca']]
usuarios_conversion = events.loc[events['event'] == 'conversion',['person','model','marca']] 

In [141]:
usuarios_checkout.rename(columns={'marca':'marca_checkout'},inplace=True)
usuarios_checkout.head()

,person,model,marca_checkout
33,15ea8012,Samsung Galaxy S8 Plus,Samsung
60,43790d8f,Motorola Moto G4 Plus,Motorola
76,d614c608,Samsung Galaxy J5,Samsung
133,55d1e0ee,Samsung Galaxy Core Plus Duos TV,Samsung
147,bb78c182,Motorola Moto G3 4G,Motorola


In [142]:
usuarios_conversion.rename(columns={'marca':'marca_conversion'},inplace=True)
usuarios_conversion.head()

,person,model,marca_conversion
195,49c19e32,iPhone 5c,iPhone
234,49c19e32,iPhone 5c,iPhone
724,39df97e0,Samsung Galaxy J2 Prime TV,Samsung
744,380c0e60,Samsung Galaxy Note 5,Samsung
1198,35ee0cc8,Samsung Galaxy A5 2017,Samsung


In [143]:
system_check_vs_conver = pd.merge(usuarios_checkout,usuarios_conversion,on = 'person', how = 'inner')

In [144]:
system_check_vs_conver.drop_duplicates(inplace = True)
system_check_vs_conver.drop(columns=['model_x','model_y'],inplace=True)
system_check_vs_conver.head()

,person,marca_checkout,marca_conversion
0,15ea8012,Samsung,Samsung
10,15ea8012,Samsung,Samsung
20,55d1e0ee,Samsung,Samsung
24,02a9fd4c,iPhone,Samsung
26,02a9fd4c,Samsung,Samsung


In [145]:
system_check_vs_conver['coincide'] = system_check_vs_conver['marca_checkout'] == system_check_vs_conver['marca_conversion'] 
system_check_vs_conver['coincide'] = np.where(system_check_vs_conver['coincide'] == True,1,0)
system_check_vs_conver.head()

,person,marca_checkout,marca_conversion,coincide
0,15ea8012,Samsung,Samsung,1
10,15ea8012,Samsung,Samsung,1
20,55d1e0ee,Samsung,Samsung,1
24,02a9fd4c,iPhone,Samsung,0
26,02a9fd4c,Samsung,Samsung,1


In [146]:
system_check_vs_conver = system_check_vs_conver.groupby('person')['coincide'].mean().to_frame()

In [147]:
system_check_vs_conver.reset_index(inplace=True)

In [148]:
system_check_vs_conver.head()

,person,coincide
0,000ba417,0.333333
1,001001be,1.000000
2,001804a2,1.000000
3,0019e639,0.375000
4,001b0bf9,1.000000


In [149]:
system_check_vs_conver.to_csv('features_system.csv', index=False)